In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch.distributions.multivariate_normal import MultivariateNormal
import matplotlib.pyplot as plt

In [2]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [3]:
train_data = pd.read_csv('Train.csv')
img = train_data.drop(["Class"],axis = 1)
#y = train_data["Class"]
img = np.asarray(img,dtype='float')
#y = np.asarray(y,dtype='float')
img= torch.tensor(img)
#y= torch.tensor(y)

#img = sigmoid(img)


In [4]:
y.view(-1,1)

NameError: name 'y' is not defined

In [ ]:
n, m = img.size()
n

In [ ]:
means = torch.rand(2, 10)
means

In [ ]:
torch.full((img.size()[0],), 2 + 1, dtype=torch.long)

In [ ]:
class Kmeans(torch.nn.Module):
    def __init__(self, img, k):
        self.n, self.m = img.size()             #可可
        self.img = img                          #可可
        self.k = k                              #可可
        self.means = torch.rand(self.k, self.m) #可可
        self.r = torch.full((self.img.size()[0],), self.k + 1, dtype=torch.long) #可可
        
    def fit(self, max_iter = 300):
        for i in range(max_iter):
            dist = torch.sum((self.img[:, None] - self.means) ** 2, axis=2)
            new_r = torch.argmin(dist, axis=1)
            #print(new_r)
            #print(sum(new_r == 0))
            if torch.equal(self.r, new_r):
                break
            else:
                self.r = new_r
            for j in range(self.k):
                data_k = self.img[torch.nonzero(torch.where(self.r == j,torch.full_like(self.r, 1),torch.full_like(self.r, 0)), as_tuple=True)]
                if len(data_k) == 0:
                    self.means[j] = torch.rand(self.m)
                else:
                    self.means[j] = torch.mean(data_k, axis=0)
        
    #def predict(self):
        #new_data = torch.round(self.means[self.r]*255)
        #disp = Image.fromarray(new_data.view(self.m, self.n, self.channel).numpy().astype('uint8'))
        #disp.show(title='k-means')
        #disp.save('k-means(pytorch)_'+str(self.k)+'.png')

In [ ]:
k = Kmeans(img,2)
k.fit()
#k.predict()

In [ ]:
k.means

In [ ]:
k.r

In [5]:
def cov(m, rowvar=False):
    if m.dim() > 2:
        raise ValueError('m has more than 2 dimensions')
    if m.dim() < 2:
        m = m.view(1, -1)
    if not rowvar and m.size(0) != 1:
        m = m.t()
    # m = m.type(torch.double)  # uncomment this line if desired
    fact = 1.0 / (m.size(1) - 1)
    m -= torch.mean(m, dim=1, keepdim=True)
    mt = m.t()  # if complex: mt = m.t().conj()
    return fact * m.matmul(mt).squeeze()

In [6]:
class GaussianMixture(torch.nn.Module):
    def __init__(self, img, k):
        super(GaussianMixture, self).__init__()
        self.n, self.m= img.size()
        #self.total_length = self.m*self.n
        self.img = img
        self.k = k
        self.K_means = Kmeans(img,k)
        self.K_means.fit()
        self.rnk = self.K_means.r
        self.means = self.K_means.means
        
    def _init_parms(self):
        self.pi = torch.tensor([torch.nonzero(torch.where(self.rnk == i,torch.full_like(self.rnk, 1),torch.full_like(self.rnk, 0)), as_tuple=True)[0].size(0)/float(self.n) for i in range(self.k)])
        self.Cov = torch.tensor([cov(k.img[torch.nonzero(torch.where(self.rnk == i,torch.full_like(self.rnk, 1),torch.full_like(self.rnk, 0)), as_tuple=True)]).detach().numpy() for i in range(self.k)])      
        #print(self.Cov.size())
        #print(self.means.size())
        #print(self.img.size())
        self.psb = torch.exp(torch.tensor([MultivariateNormal(self.means[i],self.Cov[i]).log_prob(self.img).detach().numpy() for i in range(self.k)]).T)*self.pi      
        #print(self.psb) 
                
        
        
    def fit(self, max_iter = 100, delta = 1e-3):
        likelihood = []        
        self._init_parms()      
        log_likelihood_old = self.log_likelihood().item()
        likelihood.append(log_likelihood_old)
        i = 1
        j = np.inf
        while i<= max_iter or j >= delta:
            self.E_step()
            self.M_step()
            log_likelihood = self.log_likelihood()
            i += 1
            j = log_likelihood - log_likelihood_old
            log_likelihood_old = log_likelihood
            likelihood.append(log_likelihood_old)
            
        r = torch.argmax(self.psb, axis=1)
        plt.plot(range(len(likelihood)),likelihood)
        plt.title('GMM maximum likelihood curve')
        plt.xlabel('iterations')
        plt.ylabel('log p(x)')
        plt.show()
        print(r)
        #new_data = torch.round(self.means[r]*255)
        #disp2 = Image.fromarray(new_data.view(self.m, self.n, self.channel).numpy().astype('uint8'))
        #disp2.show(title='GMM')
        #disp2.save('GMM_'+str(self.k)+'.png')
        
    def E_step(self):
        self.beta = self.psb/torch.sum(self.psb, axis=1).view(-1,1)
    
    def M_step(self):
        N = torch.sum(self.beta, axis=0)
        self.means = torch.sum(self.img[:, None] * self.beta[:, :, None], axis=0)/N[:, None]
        #self.Cov = torch.tensor([(((self.img - self.means[j]) * self.beta[:, j, None]).T.matmul(self.img - self.means[j])/N[j]).detach().numpy() for j in range(self.k)])        
        for j in range(self.k):
            self.Cov[j] = ((self.img - self.means[j]) * self.beta[:, j, None]).T.matmul(self.img - self.means[j])/N[j]
        self.pi = N/self.n
        #self.psb = torch.exp(torch.tensor([MultivariateNormal(self.means[i],self.Cov[i]).log_prob(self.img).detach().numpy() for i in range(self.k)]).T)*self.pi
        for j in range(self.k):
            try:
                self.psb[:, j] = torch.exp(MultivariateNormal(self.means[j],self.Cov[j]).log_prob(self.img))*self.pi[j]
            except :
                self.means[j] = torch.rand(self.m)
                temp = torch.rand(self.m, self.m)
                self.Cov[j] = temp.matmul(temp.T)
                self.psb[:, j] = torch.exp(MultivariateNormal(self.means[j],self.Cov[j]).log_prob(self.img))*self.pi[j]
    def log_likelihood(self):
        li = torch.tensor([MultivariateNormal(self.means[i],self.Cov[i]).log_prob(self.img).detach().numpy() for i in range(self.k)]).T + torch.log(self.pi)
        return torch.logsumexp(li, dim=1).sum()

In [214]:
gmm = GaussianMixture(img,2) 
gmm.fit()

ValueError: The parameter covariance_matrix has invalid values